# USING SLM-API TO FETCH IC DATA--##########

In [1]:
import requests
from requests_oauth2client import *
from requests_oauthlib import OAuth2

client_id =  'Aura_IC_Creation_KV'             
client_secret = 'vivek' 
url="https://cloudsso.cisco.com/as/token.oauth2"
r=requests.post(url,auth=(client_id, client_secret),data={"grant_type":"password","clientid":client_id,"clientsecret":client_secret,"scope":"profile email","username":"kavivek","password":""},headers={'Content-type':'application/x-www-form-urlencoded'})
token=r.json()['access_token']
print(token)


ModuleNotFoundError: No module named 'requests_oauth2client'

In [4]:
import requests
from requests_oauth2client import BearerAuth

resp = requests.get("https://api-supplychain.cisco.com/pdafapp/common/1.0/getMPBugDetails?osType=N&release=7.3(1)D1(1)&prodFamily=Cisco Nexus 7000 Series Switches&limit=15", auth=BearerAuth('G73tA8YTmdnyDJZlcPeKGQuMHSFf'))



ModuleNotFoundError: No module named 'requests_oauth2client'

In [3]:
import json
from json import JSONDecodeError
def get_bugid_records(text):
    result=[]
    lines=text.split("\n")
    c=0
    for line in lines:
        line=line.strip("[],")
        if(line!=""):
            try:
                record=json.loads(line)
                result.append(record)
            except JSONDecodeError:
                c=c+1 
    print("fault lines:",c)
    return result

In [5]:
import pandas as pd

api_inputs_df=pd.read_excel("./data/MP_Details_Jan7.xlsx")     #"./data/MP_ios_xr_nexus_oct_8th_2021.xlsx"
projids=pd.read_excel("./data/MP_ios_xr_nexus_oct_8th_2021.xlsx")["Master Project ID"].values

In [6]:
len(projids)

105

In [8]:
ostype_dict={"nexus":"N","ios-xr":"X","ios/xe":"I","n":"N","i":"I","x":"X","aci":"ACI","sdwan":"vEdge","ucs":"UCS"}
cols=['PROJID','BUGID','OSTYPE','RELEASE','PRODFAMILY','SELECTIONSTATUS', 'HARDWARE', 'IMPACTCATEGORYOTHER', 'ARCHITECTURE', 'FEATURE', 'TRIGGERVALUE', 'IMPACTDESC','TRIGGERREPRO', 'OSPLATFORM', 'SYMPTOMS', 'WORKAROUNDAVOIDANCE', 'WORKAROUNDRECOVERY', 'OTHER', 'ISMANUALPUBLISH','CONDITIONS', 'IMPACTCATEGORY', 'FOUND', 'TECHNOLOGY', 'APPLICABLERELEASE', 'REVIEWTYPE', 'PUBLISHEDDATE','ISPUBLISHED', 'CLI', 'FIXEDIN', 'COMMENTS', 'WORKAROUNDAVAILABLE', 'IMPACTEDPLATFORMS', 'SUMMARY']


mp_data_df_list=[]
for i in range(len(api_inputs_df)):
    mp_data_temp=pd.DataFrame(columns=cols)
    mp_data_df_list.append(mp_data_temp)
    
for i in range(len(api_inputs_df)):
    print(i)
    projid=api_inputs_df["Master Project ID"].iloc[i]
    if(projid in projids):  #not
        ostype=ostype_dict[api_inputs_df["OS Type"].iloc[i].lower()]
        rel=api_inputs_df["Base Release"].iloc[i]
        if(ostype=='X'):rel="BASE"+rel
        prodf=api_inputs_df["PF Information"].iloc[i]
        temp=prodf.split(",")
        if(len(temp)>1):prodf='/'+temp[0]
        limit="0"
        url="https://api-supplychain.cisco.com/pdafapp/common/1.0/getMPBugDetails?osType="+ostype+"&release="+rel+"&prodFamily="+prodf+"&limit="+limit
        resp = requests.get(url, auth=BearerAuth(token))
        text=resp.text
        print(i, text[:100])
        records=get_bugid_records(text)
        for record in records:
            l=[]
            for col in cols:l.append(record[col])
            mp_data_df_list[i].loc[len(mp_data_df_list[i])]=l
    print("\n\n")
    
    


0
0 [{"ISPUBLISHED":"Y","CLI":null,"FIXEDIN":null,"COMMENTS":null,"WORKAROUNDAVAILABLE":null,"IMPACTEDPL
fault lines: 1



1
1 [{"ISPUBLISHED":"Y","CLI":null,"FIXEDIN":null,"COMMENTS":null,"WORKAROUNDAVAILABLE":null,"IMPACTEDPL
fault lines: 1



2
2 [{"ISPUBLISHED":"Y","CLI":"^feature vrrp","FIXEDIN":null,"COMMENTS":null,"WORKAROUNDAVAILABLE":"No",
fault lines: 2



3
3 [{"ISPUBLISHED":"Y","CLI":null,"FIXEDIN":"7.0(3)I7(6)","COMMENTS":null,"WORKAROUNDAVAILABLE":"Yes","
fault lines: 0



4
4 [{"ISPUBLISHED":"Y","CLI":null,"FIXEDIN":null,"COMMENTS":null,"WORKAROUNDAVAILABLE":null,"IMPACTEDPL
fault lines: 1



5
5 []
fault lines: 0



6
6 [{"ISPUBLISHED":"Y","CLI":null,"FIXEDIN":null,"COMMENTS":null,"WORKAROUNDAVAILABLE":null,"IMPACTEDPL
fault lines: 1



7
7 [{"ISPUBLISHED":"Y","CLI":"~ipv6","FIXEDIN":"8.2(2)","COMMENTS":null,"WORKAROUNDAVAILABLE":"No","IMP
fault lines: 0



8
8 [{"ISPUBLISHED":"Y","CLI":null,"FIXEDIN":null,"COMMENTS":null,"WORKAROUNDAVAILABLE":null,"IMPACTEDPL
fault lin

In [16]:
mp_ic_data=pd.concat(mp_data_df_list)
print(len(mp_ic_data))

353703


In [17]:
mp_ic_data.to_excel("./MP_Bugs_Data/MPs_IC_mar14.xlsx",index=False)

In [ ]:
###########3-------------------END OF COLLECTING BUG RECORDS PRESENT ACROSS ALL GIVEN MASTER PROJECT IDS-------####

# ----FIGURE OUT NEW RECORDS from jan18th to marth14----###

In [18]:
import pandas as pd

mp_data1=pd.read_excel("./MP_Bugs_Data/MPs_IC_part1_dec9.xlsx")
mp_data2=pd.read_excel("./MP_Bugs_Data/MPs_IC_part2_jan18.xlsx")
mp_data3=pd.read_excel("./MP_Bugs_Data/MPs_IC_part3_mar14.xlsx")

mp_data=pd.concat([mp_data1,mp_data2,mp_data3])


In [21]:
mp_data1=mp_data[['PROJID','BUGID']]

In [22]:
mp_data_df_new_records = mp_ic_data.merge(mp_data1, on=['PROJID','BUGID'],how='left', indicator=True)
mp_data_df_new_records = mp_data_df_new_records[mp_data_df_new_records['_merge'] == 'left_only']


In [23]:
print(mp_data_df_new_records.columns)
print(len(mp_data_df_new_records))
print(len(mp_ic_data))

Index(['PROJID', 'BUGID', 'OSTYPE', 'RELEASE', 'PRODFAMILY', 'SELECTIONSTATUS',
       'HARDWARE', 'IMPACTCATEGORYOTHER', 'ARCHITECTURE', 'FEATURE',
       'TRIGGERVALUE', 'IMPACTDESC', 'TRIGGERREPRO', 'OSPLATFORM', 'SYMPTOMS',
       'WORKAROUNDAVOIDANCE', 'WORKAROUNDRECOVERY', 'OTHER', 'ISMANUALPUBLISH',
       'CONDITIONS', 'IMPACTCATEGORY', 'FOUND', 'TECHNOLOGY',
       'APPLICABLERELEASE', 'REVIEWTYPE', 'PUBLISHEDDATE', 'ISPUBLISHED',
       'CLI', 'FIXEDIN', 'COMMENTS', 'WORKAROUNDAVAILABLE',
       'IMPACTEDPLATFORMS', 'SUMMARY', '_merge'],
      dtype='object')
10605
353703


In [24]:
mp_data_df_new_records.to_excel("./MP_Bugs_Data/MPs_IC_part4_may10.xlsx",index=False)